In [1]:
import pandas as pd
import html5lib
import time
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import mplfinance as mpf
from datetime import datetime, timedelta,date
import datetime
import plotly
import plotly.graph_objs as go
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from forex_python.converter import CurrencyRates

In [48]:
class Manage_share_database():
    
    def __init__(self):
                
        self.location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V2.sqlite'
        
    def get_symbol_id(self,market):
        
        con = sqlite3.connect(self.location_db)
        cur = con.cursor()
        
        sql = f"""select Information.Symbol,Information.SymbolId  
                from Information Inner join Market on Market.MarketId = Information.MarketId
                where Market.Mname = "{market}"
                """

        cur.execute(sql)
        records = cur.fetchall()
        con.close()
        symbol_id = {}
        
        for share in records:
            
            symbol_id[share[0]] = share[1]
        
        return symbol_id
    
    # ดึงราคหุ้นจาก yfinance รายวัน
    def updateData_Day(self,ticker,market):

        if ticker in self.get_symbol_id(market).keys():
            
            start_date = self.get_max_date('Stock_price_day',self.get_symbol_id(market)[ticker]) + datetime.timedelta(days=1)
            
            end_date = (datetime.datetime.now()).strftime('%Y-%m-%d')

            if start_date < (datetime.datetime.strptime(end_date, '%Y-%m-%d')) :

                detail = yf.download(ticker, interval='1D', start=start_date, end=end_date)

                if not(detail.empty):
                    detail['SymbolId'] = str(int(self.get_symbol_id(market)[ticker]))
                    table_price = detail
                    table_price = detail.reset_index()

                    return table_price

        return pd.DataFrame({'Date':[],'Open':[],'High':[],'Low':[],'Close':[],'Adj Close':[],'Volume':[],'SymbolId':[]})
    
    # ดึงราคหุ้นจาก yfinance รายชั่วโมง
    def updateData_hours(self,ticker,market):


        if ticker in self.get_symbol_id(market).keys():
            
            max_date = self.get_max_date('Stock_price_hours',self.get_symbol_id(market)[ticker])
            start_date =  max_date + datetime.timedelta(hours=1)
       
            detail = yf.download(ticker, interval='1h', start=start_date )

            if not(detail.empty):

                detail.index = detail.index.tz_convert('Asia/Bangkok')
                detail.index = detail.index.tz_localize(None)
                detail['SymbolId'] = int(self.get_symbol_id(market)[ticker])
                table_price = detail.reset_index()
            
                return table_price
          
        return pd.DataFrame({'Datetime':[],'Open':[],'High':[],'Low':[],'Close':[],'Adj Close':[],'Volume':[],'SymbolId':[]})

    # นำ dataframe มาเพิ่มลงใน database 
    def combineData(self,stock_name,table_price):
        
        try :
            
            if table_price.empty:
                return ''
            
            con = sqlite3.connect(self.location_db)
            cur = con.cursor()
            column = {'Stock_price_hours':'Datetime','Stock_price_day':'Date'}
            sql = """ select * from {} where {} = "{}" and SymbolId = "{}" """.format(stock_name,column[stock_name],table_price.iloc[0][column[stock_name]],int(table_price.iloc[0]['SymbolId']))
            cur.execute(sql)
            con.close
            records = cur.fetchall()

            if records == []:

                table_price[column[stock_name]] = pd.to_datetime(table_price[column[stock_name]], format='%Y/%m/%d %H:%M:%S')
                new_table = table_price.to_sql(stock_name,con,index=False,if_exists='append')
                con.commit()
                con.close()
                return 'Done Update : ',new_table

            else :

                return 'Data already in database'

        except KeyError:
            
            return 'Table name incorrect'
        
    def get_max_date(self,table,share):
    
        
        try :
            
            con = sqlite3.connect(self.location_db)
            cur = con.cursor()
            
            column = {'Stock_price_hours':'Datetime','Stock_price_day':'Date'}

            sql = """select {} From {} where SymbolId = '{}' ORDER BY {} DESC LIMIT 1;
            """.format(column[table],table,share,column[table])

            stock_data = pd.read_sql(sql,con)
            con.close()

            stock_data[column[table]] = pd.to_datetime(stock_data[column[table]], format='%Y/%m/%d')

            return max(stock_data[column[table]])
        
        except ValueError: 
            reverse = {'Date' : 360*50,'Datetime' : 720}
            stock_data = pd.DataFrame(['0'],columns =[column[table]])
            stock_data[column[table]][0] = pd.to_datetime(date.today() - timedelta(days=reverse[column[table]]), format='%Y/%m/%d %H:%M:%S')
            return max(stock_data[f'{column[table]}'])
    
          
# Update Statistics--------------------------------------------------

    def Check_Statistics(self,table,df,time):

        location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\Set_dataframe_to_sqlite3\share.sqlite'
        con = sqlite3.connect(location_db)
        cur = con.cursor()

        sql = """select "{}" From {} where SymbolId = '{}' and "{}" = '{}' LIMIT 1
        """.format("Statistics as of",table,int(df.iloc[[0]]['SymbolId']),
                   "Statistics as of",str(time))

        cur.execute(sql)
        records = cur.fetchall()
        con.close()

        return not(bool(records))

    def new_Statistics(self,Statistics_df):
        data_new_year = []
        r = 0
        for time in (Statistics_df['Statistics as of'].tolist())[:-1]:

            Nhave = self.Check_Statistics('Satistics_year',Statistics_df.iloc[[r]],time)

            if Nhave :

                data_new_year.append(Statistics_df.iloc[[r]])

            r+=1

        if data_new_year :

            return pd.concat(data_new_year)

        return pd.DataFrame({'Statistics as of': []
            , 'Last Price (Baht)': []
            , 'Market Cap. (M.Baht)': []
            , 'F/S Period (As of date)': []
            , 'P/E': [], 'P/BV': []
            , 'Book Value per share (Baht)': [], 'Dividend Yield (%)': []
            , 'SymbolId': []})

#-----------------------------------------------------------------------------------------



In [49]:
tool = Manage_share_database()

In [23]:
# pd.to_datetime(date.today() - timedelta(days=730), format='%Y/%m/%d %H:%M:%S')
tool.get_max_date('Stock_price_day','609')

Timestamp('2023-01-17 00:00:00')

In [6]:
pd.to_datetime(date.today() - timedelta(days=729), format='%Y/%m/%d %H:%M:%S')

Timestamp('2021-03-04 00:00:00')

In [11]:
tool.get_symbol_id("SET")

{'EE.BK': 1,
 'GFPT.BK': 2,
 'LEE.BK': 3,
 'MAX.BK': 4,
 'NER.BK': 5,
 'PPPM.BK': 6,
 'STA.BK': 7,
 'TEGH.BK': 8,
 'TFM.BK': 9,
 'TRUBB.BK': 10,
 'TWPC.BK': 11,
 'UPOIC.BK': 12,
 'UVAN.BK': 13,
 'VPO.BK': 14,
 'AAI.BK': 15,
 'APURE.BK': 16,
 'ASIAN.BK': 17,
 'BR.BK': 18,
 'BRR.BK': 19,
 'BTG.BK': 20,
 'CBG.BK': 21,
 'CFRESH.BK': 22,
 'CH.BK': 23,
 'CHOTI.BK': 24,
 'CM.BK': 25,
 'CPF.BK': 26,
 'CPI.BK': 27,
 'F&D.BK': 28,
 'GLOCON.BK': 29,
 'HTC.BK': 30,
 'ICHI.BK': 31,
 'ITC.BK': 32,
 'JDF.BK': 33,
 'KBS.BK': 34,
 'KSL.BK': 35,
 'KTIS.BK': 36,
 'LST.BK': 37,
 'M.BK': 38,
 'MALEE.BK': 39,
 'MINT.BK': 40,
 'NRF.BK': 41,
 'NSL.BK': 42,
 'OISHI.BK': 43,
 'OSP.BK': 44,
 'PB.BK': 45,
 'PLUS.BK': 46,
 'PM.BK': 47,
 'PRG.BK': 48,
 'RBF.BK': 49,
 'SAPPE.BK': 50,
 'SAUCE.BK': 51,
 'SFP.BK': 52,
 'SNNP.BK': 53,
 'SNP.BK': 54,
 'SORKON.BK': 55,
 'SSC.BK': 56,
 'SSF.BK': 57,
 'SST.BK': 58,
 'SUN.BK': 59,
 'TC.BK': 60,
 'TFG.BK': 61,
 'TFMAMA.BK': 62,
 'TIPCO.BK': 63,
 'TKN.BK': 64,
 'TU.BK': 65,
 '

---------------------------------------------------SET100--------------------------------------------------------------------

In [12]:
tickers = ['ACE.BK', 'ADVANC.BK', 'AEONTS.BK', 'AMATA.BK', 'AOT.BK', 'AP.BK', 'AWC.BK', 'BAM.BK',
       'BANPU.BK', 'BBL.BK', 'BCH.BK', 'BCP.BK', 'BCPG.BK', 'BDMS.BK', 'BEC.BK', 'BEM.BK',
       'BGRIM.BK', 'BH.BK', 'BLA.BK', 'BTS.BK', 'CBG.BK', 'CENTEL.BK', 'CHG.BK', 'CK.BK', 'CKP.BK',
       'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK', 'DOHOME.BK', 'DTAC.BK', 'EA.BK',
       'EGCO.BK', 'EPG.BK', 'ESSO.BK', 'FORTH.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK', 'GUNKUL.BK',
       'HANA.BK', 'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'JMART.BK', 'JMT.BK', 'KBANK.BK',
       'KCE.BK', 'KEX.BK', 'KKP.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'MAJOR.BK', 'MEGA.BK', 'MINT.BK',
       'MTC.BK', 'ONEE.BK', 'OR.BK', 'ORI.BK', 'OSP.BK', 'PLANB.BK', 'PSL.BK', 'PTG.BK', 'PTT.BK',
       'PTTEP.BK', 'PTTGC.BK', 'QH.BK', 'RATCH.BK', 'RBF.BK', 'RCL.BK', 'SAWAD.BK', 'SCB.BK',
       'SCC.BK', 'SCGP.BK', 'SINGER.BK', 'SPALI.BK', 'SPRC.BK', 'STA.BK', 'STARK.BK', 'STEC.BK',
       'STGT.BK', 'SUPER.BK', 'SYNEX.BK', 'TASCO.BK', 'TCAP.BK', 'THANI.BK', 'TIDLOR.BK',
       'TIPH.BK', 'TISCO.BK', 'TOP.BK', 'TQM.BK', 'TRUE.BK', 'TTA.BK', 'TTB.BK', 'TU.BK', 'VGI.BK',
       'WHA.BK']

In [13]:
tool.updateData_Day('ACE.BK','SET')

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,SymbolId
0,2023-01-18,2.66,2.70,2.66,2.66,2.66,22458900,374
1,2023-01-19,2.68,2.72,2.66,2.68,2.68,27451100,374
2,2023-01-20,2.68,2.68,2.64,2.66,2.66,18616500,374
3,2023-01-23,2.66,2.66,2.64,2.64,2.64,14109400,374
4,2023-01-24,2.66,2.66,2.64,2.64,2.64,16344300,374
5,2023-01-25,2.64,2.66,2.60,2.62,2.62,26715600,374
6,2023-01-26,2.62,2.64,2.58,2.58,2.58,22764700,374
7,2023-01-27,2.60,2.60,2.56,2.56,2.56,18930100,374
8,2023-01-30,2.56,2.58,2.54,2.56,2.56,24377600,374
9,2023-01-31,2.56,2.60,2.56,2.58,2.58,11311600,374


In [25]:
tool.updateData_hours('WHA.BK',"SET")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WHA.BK: No data found for this date range, symbol may be delisted


,Datetime,Open,High,Low,Close,Adj Close,Volume,SymbolId


In [26]:
new = []
for ticker in tickers:
    
    print(ticker)
    df = tool.updateData_hours(ticker,"SET")
    new.append(df)
    # tool.combineData('Stock_price_hours',df)
# new_stock = pd.concat(new)

ACE.BK
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACE.BK: No data found for this date range, symbol may be delisted
ADVANC.BK
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ADVANC.BK: No data found for this date range, symbol may be delisted
AEONTS.BK
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AEONTS.BK: No data found for this date range, symbol may be delisted
AMATA.BK
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AMATA.BK: No data found for this date range, symbol may be delisted
AOT.BK
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AOT.BK: No data found for this date range, symbol may be delisted
AP.BK
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AP.BK: No data found for this date range, symbol may be delisted
AW

In [36]:
new = []
for ticker in tickers:
    
    print(ticker)
    df = tool.updateData_Day(ticker,"SET")
    new.append(df)
    tool.combineData('Stock_price_day',df)
new_stock = pd.concat(new)

ACE.BK
[*********************100%***********************]  1 of 1 completed

                    select * 
                    from Stock_price_day
                    where Date = "2023-01-18 00:00:00" and SymbolId = "374"

                    
ADVANC.BK
[*********************100%***********************]  1 of 1 completed

                    select * 
                    from Stock_price_day
                    where Date = "2023-01-18 00:00:00" and SymbolId = "575"

                    
AEONTS.BK
[*********************100%***********************]  1 of 1 completed

                    select * 
                    from Stock_price_day
                    where Date = "2023-01-18 00:00:00" and SymbolId = "609"

                    
AMATA.BK
[*********************100%***********************]  1 of 1 completed

                    select * 
                    from Stock_price_day
                    where Date = "2023-01-18 00:00:00" and SymbolId = "225"

                    
AOT.BK
[

In [35]:
# new_stock.to_csv('save_test.csv')
new_stock 

,Date,Open,High,Low,Close,Adj Close,Volume,SymbolId
0,2023-01-18,2.66,2.70,2.66,2.66,2.66,22458900,374
1,2023-01-19,2.68,2.72,2.66,2.68,2.68,27451100,374
2,2023-01-20,2.68,2.68,2.64,2.66,2.66,18616500,374
3,2023-01-23,2.66,2.66,2.64,2.64,2.64,14109400,374
4,2023-01-24,2.66,2.66,2.64,2.64,2.64,16344300,374
...,...,...,...,...,...,...,...,...
27,2023-02-24,4.00,4.02,3.94,3.98,3.98,181460000,281
28,2023-02-27,4.00,4.08,3.94,4.06,4.06,145975100,281
29,2023-02-28,4.06,4.06,3.92,3.94,3.94,231034600,281
30,2023-03-01,3.94,4.02,3.92,3.94,3.94,180614400,281


------------------------------NASDAQ--------------------------------------------------------------------------------

In [37]:
triker_NASDAQ = ['AAPL','MSFT','GOOG','GOOGL',
 'AMZN',
 'PCAR',
 'BRK/A',
 'BRK/B',
 'TSLA',
 'NVDA',
 'V',
 'TSM',
 'UNH',
 'XOM',
 'META',
 'JNJ',
 'JPM',
 'WMT',
 'MA',
 'PG',
 'HD',
 'NVO',
 'CVX',
 'LLY',
 'BAC',
 'MRK',
 'ABBV',
 'BABA',
 'ASML',
 'KO',
 'AVGO',
 'PEP',
 'PFE',
 'ORCL',
 'COST',
 'TMO',
 'AZN',
 'SHEL',
 'CSCO',
 'MCD',
 'TM',
 'NKE',
 'DIS',
 'DHR',
 'ABT',
 'NVS',
 'TMUS',
 'WFC',
 'ACN',
 'VZ',
 'BHP',
 'MS',
 'CRM',
 'CMCSA',
 'ADBE',
 'TXN',
 'UPS',
 'LIN',
 'PM',
 'NFLX',
 'TTE',
 'NEE',
 'SCHW',
 'BMY',
 'HSBC',
 'RTX',
 'RY',
 'QCOM',
 'T',
 'SAP',
 'HON',
 'AXP',
 'UL',
 'HDB',
 'DE',
 'LOW',
 'AMGN',
 'CAT',
 'AMD',
 'COP',
 'BA',
 'TD',
 'GS',
 'UNP',
 'SBUX',
 'IBM',
 'LMT',
 'RIO',
 'BP',
 'SNY',
 'SPGI',
 'ELV',
 'PDD',
 'BUD',
 'PLD',
 'INTC',
 'CVS',
 'INTU',
 'MDT',
 'BLK',
 'GILD',
 'SONY',
 'SYK',
 'C',
 'DEO',
 'AMT',
 'ADI',
 'AMAT',
 'EQNR',
 'BKNG',
 'ADP',
 'TJX',
 'MDLZ',
 'GE',
 'EL',
 'CI',
 'NOW',
 'MUFG',
 'CB',
 'MO',
 'BTI',
 'PYPL',
 'ISRG',
 'JD',
 'PGR']

In [47]:
new = []
for ticker in triker_NASDAQ:
    print(ticker)
    df = tool.updateData_Day(ticker,"NASDAQ")
    new.append(df)
    # tool.combineData('Stock_price_day',df)
new_stock = pd.concat(new)

AAPL
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, SymbolId]
Index: []
MSFT
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, SymbolId]
Index: []
GOOG
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, SymbolId]
Index: []
GOOGL
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, SymbolId]
Index: []
AMZN
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, SymbolId]
Index: []
PCAR
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, SymbolId]
Index: []
BRK/A
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, SymbolId]
Index: []
BRK/B
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, SymbolId]
Index: []
TSLA
[*********************100%***********************]  1 of 1 completed
           Date        Open        High         Low       Close   Adj Close  \
0    2010-06-29    1.266667    1.666667    1

In [39]:
new_stock

,Date,Open,High,Low,Close,Adj Close,Volume,SymbolId
0,1980-12-12 00:00:00,0.128348,0.128906,0.128348,0.128348,0.099722,469033600.0,636
1,1980-12-15 00:00:00,0.122210,0.122210,0.121652,0.121652,0.094519,175884800.0,636
2,1980-12-16 00:00:00,0.113281,0.113281,0.112723,0.112723,0.087582,105728000.0,636
3,1980-12-17 00:00:00,0.115513,0.116071,0.115513,0.115513,0.089749,86441600.0,636
4,1980-12-18 00:00:00,0.118862,0.119420,0.118862,0.118862,0.092351,73449600.0,636
...,...,...,...,...,...,...,...,...
2205,2023-02-24 00:00:00,44.737999,45.564999,44.455002,44.799999,44.799999,11854300.0,2721
2206,2023-02-27 00:00:00,45.500000,45.500000,44.490002,44.740002,44.740002,8018000.0,2721
2207,2023-02-28 00:00:00,44.275002,45.110001,43.970001,44.439999,44.439999,6437600.0,2721
2208,2023-03-01 00:00:00,46.540001,46.770000,45.299999,45.730000,45.730000,9294900.0,2721


In [50]:
new = []
for ticker in triker_NASDAQ:
    print(ticker)
    df = tool.updateData_hours(ticker,"NASDAQ")
    new.append(df)
    # tool.combineData('Stock_price_hours',df)
new_stock = pd.concat(new)

AAPL
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-15 20:30:00" and SymbolId = "636" 
MSFT
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-15 20:30:00" and SymbolId = "3203" 
GOOG
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-15 20:30:00" and SymbolId = "2307" 
GOOGL
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-15 20:30:00" and SymbolId = "2308" 
AMZN
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-15 20:30:00" and SymbolId = "873" 
PCAR
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-15 20:30:00" and Symbol

In [51]:
new_stock

,Datetime,Open,High,Low,Close,Adj Close,Volume,SymbolId
0,2021-03-15 20:30:00,121.260002,123.569901,120.419998,122.598396,122.598396,25375805.0,636.0
1,2021-03-15 21:30:00,122.599701,123.230003,122.235001,122.389999,122.389999,11839733.0,636.0
2,2021-03-15 22:30:00,122.389999,122.489998,122.040001,122.334999,122.334999,7767154.0,636.0
3,2021-03-15 23:30:00,122.339996,122.720001,122.239998,122.680099,122.680099,5902142.0,636.0
4,2021-03-16 00:30:00,122.680000,123.639999,122.650002,123.480003,123.480003,8986442.0,636.0
...,...,...,...,...,...,...,...,...
3459,2023-03-02 23:30:00,46.540001,46.720001,46.410000,46.560001,46.560001,675334.0,2721.0
3460,2023-03-03 00:30:00,46.560001,46.650002,46.529999,46.599998,46.599998,513631.0,2721.0
3461,2023-03-03 01:30:00,46.599998,46.830002,46.564999,46.779999,46.779999,490251.0,2721.0
3462,2023-03-03 02:30:00,46.779999,47.020000,46.689999,47.000000,47.000000,867705.0,2721.0


In [199]:
new_stock.to_csv('save_test.csv')

In [307]:
# tool.combineData('Stock_price_hours',new_stock)


                    select * 
                    from Stock_price_hours
                    where Datetime = "2021-03-04 21:30:00" and SymbolId = "636"

                    


('Done Update : ', 131421)

-----------------------------------------CRYPTO-------------------------------------------------------------------------

In [52]:
cp = list(tool.get_symbol_id("CRYPTO").keys())[:101]

In [56]:
new = []
for ticker in cp :
    print(ticker)
    df = tool.updateData_Day(ticker,"CRYPTO")
    tool.combineData('Stock_price_day',df)
    new.append(df)

new_stock = pd.concat(new)

BTC-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_day where Date = "2014-09-17 00:00:00" and SymbolId = "4911" 
ETH-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_day where Date = "2017-11-09 00:00:00" and SymbolId = "4912" 
USDT-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_day where Date = "2017-11-09 00:00:00" and SymbolId = "4913" 
BNB-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_day where Date = "2017-11-09 00:00:00" and SymbolId = "4914" 
USDC-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_day where Date = "2018-10-08 00:00:00" and SymbolId = "4915" 
XRP-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_day where Date = "2017-11-09 00:00:00" and SymbolId = "4916" 
AD

In [59]:
new = []
for ticker in cp :
    print(ticker)
    df = tool.updateData_hours(ticker,"CRYPTO")
    tool.combineData('Stock_price_hours',df)
    new.append(df)

new_stock = pd.concat(new)

BTC-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-13 08:00:00" and SymbolId = "4911" 
ETH-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-13 08:00:00" and SymbolId = "4912" 
USDT-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-13 08:00:00" and SymbolId = "4913" 
BNB-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-13 08:00:00" and SymbolId = "4914" 
USDC-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-13 08:00:00" and SymbolId = "4915" 
XRP-USD
[*********************100%***********************]  1 of 1 completed
 select * from Stock_price_hours where Datetime = "2021-03-13

In [58]:
new_stock 

,Datetime,Open,High,Low,Close,Adj Close,Volume,SymbolId
0,2021-03-13 08:00:00,56771.054688,56771.054688,56217.972656,56485.968750,56485.968750,0,4911
1,2021-03-13 09:00:00,56470.945312,56623.253906,56242.578125,56568.539062,56568.539062,54345728,4911
2,2021-03-13 10:00:00,56563.363281,56799.175781,56411.890625,56685.136719,56685.136719,0,4911
3,2021-03-13 11:00:00,56690.183594,56820.757812,56373.281250,56820.757812,56820.757812,0,4911
4,2021-03-13 12:00:00,56817.066406,57000.589844,56705.933594,56744.835938,56744.835938,0,4911
...,...,...,...,...,...,...,...,...
9358,2023-03-03 13:00:00,0.042124,0.042124,0.041551,0.041710,0.041710,0,5011
9359,2023-03-03 14:00:00,0.041726,0.042893,0.041726,0.042865,0.042865,535712,5011
9360,2023-03-03 15:00:00,0.042885,0.043246,0.042568,0.042569,0.042569,81268,5011
9361,2023-03-03 16:00:00,0.042443,0.042891,0.042412,0.042686,0.042686,527868,5011


----------------------------------------------TEST----------------------------------------------------

In [11]:
df = yf.download('AAPL', interval='1d', start=pd.to_datetime('2023-02-17 00:00:00', format='%Y/%m/%d %H:%M:%S'))

[*********************100%***********************]  1 of 1 completed


In [12]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-17,152.350006,153.000000,150.850006,152.550003,152.550003,59095900
2023-02-21,150.199997,151.300003,148.410004,148.479996,148.479996,58772400


In [333]:
df.index = df.index.tz_convert('Asia/Bangkok')
df.index = df.index.tz_localize(None)
df = df.reset_index()

TypeError: Cannot convert tz-naive timestamps, use tz_localize to localize

In [334]:
def convert_USD_to_THA(rate,x):

    return x * rate

In [335]:
# แบบ rate ณ วันที่ทำการ update
# rate = c.convert('USD','THB' ,1)
# col = list(df.columns)
# for column in range(1,6):
#     df[f'{col[column]}'] = df.apply( lambda x: convert_USD_to_THA(rate,x[f'{col[column]}']), axis = 1)

In [336]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-17,23621.283203,857140.432454,806817.899650,844813.661379,844813.661379,1.422322e+12
2023-02-18,24565.296875,852834.616445,841469.962343,847416.159579,847416.159579,6.749205e+11
2023-02-19,24640.027344,862952.830681,836630.226492,836630.226492,836630.226492,8.788428e+11
2023-02-20,24336.623047,860456.256697,822883.385812,853877.066558,853877.066558,9.968790e+11
2023-02-21,24838.162109,864096.906246,837891.510526,847736.753289,847736.753289,9.915253e+11


In [257]:
list(df.columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SymbolId']